In [ ]:
from google.colab.auth import authenticate_user
authenticate_user()

import google.cloud.bigquery.magics
google.cloud.bigquery.magics.context.project = 'your_project_id'

! gcloud config set project { google.cloud.bigquery.magics.context.project }

Updated property [core/project].


# COVID19国内感染者数オープンデータ

厚生労働省の公開する、新型コロナウイルス感染症オープンデータ ( https://www.mhlw.go.jp/stf/covid-19/open-data.html )を用いて集計を行います。

- covid19_jp : 新規陽性者数の日別推移 を取り込んだテーブルです。
  - Date 日付
  - ALL 全国陽性者数
  - Hokkaido 北海道の陽性者数
  - ︙
  - Okinawa 沖縄の陽性者数
- covid19_jp_unpivotted 上のテーブルをunpivotしたテーブルです。ただし、0人の行は削除しています。
  - date 日付
  - prefecture 都道府県名（ローマ字）
  - new_cases 陽性者数

また、次のテーブルを使用することができます
- japan_population: 日本の都道府県別人口（令和二年国勢調査）
  - prefecture 都道府県名（ローマ字）
  - population 人口

## （一般公開にあたっての追記）オープンデータをBigQueryに取り込む方法

実際の研修では予め講師がテーブルを作成しましたが、一般公開版ではみなさまにテーブルの用意をしていただく必要があります。

1. https://www.mhlw.go.jp/stf/covid-19/open-data.html から「新規陽性者数の推移（日別）」をダウンロードします。
2. UTF-8(BOMあり)になっているので、BOMを消します。
3. BigQueryにデータセットを作成します。
4. BigQueryにアップロードします。
5. 都道府県マスタ（令和二年国勢調査より作成）のCSVをダウンロードします。
6. おなじくBigQueryにアップロードします。

以下のスクリプトを実行すると、以上の手順が行われます。（your_project_id は 書き換えて実行してください。）

In [ ]:
!wget https://covid19.mhlw.go.jp/public/opendata/newly_confirmed_cases_daily.csv
!LC_ALL=C sed -e $'1s/^\xef\xbb\xbf//' newly_confirmed_cases_daily.csv > covid19.csv
!bq mk --project_id your_project_id --dataset db_tutorial_2023
!bq load --replace --autodetect --source_format=CSV --project_id your_project_id db_tutorial_2023.covid19_jp ./covid19.csv
!wget -O pref.csv https://raw.githubusercontent.com/mixigroup/2023BeginnerTrainingDataBasePublic/master/challenge/pref.csv
!bq load --replace --autodetect --source_format=CSV --project_id your_project_id db_tutorial_2023.japan_population ./pref.csv


次に、covid19_jp_unpivottedテーブルを作成します。下のセルを実行すると作成できます。

参考: https://cloud.google.com/bigquery/docs/reference/standard-sql/query-syntax?hl=ja#unpivot_operator



In [ ]:
%%bigquery
CREATE OR REPLACE TABLE db_tutorial_2023.covid19_jp_unpivotted AS

select * from (
SELECT date, prefecture, new_cases FROM `db_tutorial_2023.covid19_jp` 
UNPIVOT(new_cases for prefecture in (Hokkaido,Aomori,Iwate,Miyagi,Akita,Yamagata,Fukushima,Ibaraki,Tochigi,Gunma,Saitama,Chiba,Tokyo,Kanagawa,Niigata,Toyama,Ishikawa,Fukui,Yamanashi,Nagano,Gifu,Shizuoka,Aichi,Mie,Shiga,Kyoto,Osaka,Hyogo,Nara,Wakayama,Tottori,Shimane,Okayama,Hiroshima,Yamaguchi,Tokushima,Kagawa,Ehime,Kochi,Fukuoka,Saga,Nagasaki,Kumamoto,Oita,Miyazaki,Kagoshima,Okinawa))
)
where new_cases > 0

以上で準備は終わりです。お疲れさまでした。

## 問1
好きな日付（2020年9月〜2023年3月、以後省略）で新規感染者数が多い順に３つ都道府県を挙げてみましょう。

### 期待する出力フォーマット

|prefecture| new_cases|
|---:|---:|
|Tokyo| 12345|
|Osaka| 9876|
|Kanagawa| 5432|

In [ ]:
%%bigquery

-- ここにクエリを書く

## 問2

好きな日付で、人口あたりの新規感染者数が多い順に３つ都道府県を挙げてみましょう

Tips: 最近のアップデートでカラム名に一部のマルチバイト文字が対応したので、試してみてもよいでしょう（まだPreview段階の機能なので注意）。
https://cloud.google.com/bigquery/docs/release-notes#March_22_2023 

### 期待する出力フォーマット

|prefecture| 百万人あたりの感染者数|
|---:|---:|
|Tokushima| 789|
|Fukuoka| 567|
|Iwate| 321|

## 問3

2021年4月1日〜30日において、新規感染者数が100人を超えていた都道府県はいくつあったか、日付別に出力しましょう。

### 期待する出力フォーマット

|date| pref_num|
|---:|---:|
|2021-04-01| 7|
|2021-04-02| 9|
|︙|︙|


## 問4
好きな日付・都道府県を選びます。

その都道府県が、その日何番目に新規感染者数が多い都道府県だったか、調べてみましょう。

出力は1レコードにしてください。

ヒント: https://cloud.google.com/bigquery/docs/reference/standard-sql/numbering_functions?hl=ja#rank

### 期待する出力フォーマット

|prefecture| new_cases| rank_number|
|---:|---:|---:|
|Ishikawa| 2085| 27|


## 問5

全期間（2020年1月〜）で、初めて感染者が出た日が一番遅かった都道府県はどこか、またその日付を調べてみましょう。

### 期待する出力フォーマット

|prefecture| first_date|
|---:|---:|
|Xxxxx| 2020-xx-xx|

## 問6

好きな1ヶ月を選びます。各日付において、その日3番目に感染者数が多い県を調べてみましょう。

### 期待する出力フォーマット

|date| pref_num|
|---:|---:|
|2022-10-01| Kanagawa|
|2022-10-02| Kanagawa|
|︙|︙|

## 問7

好きな都道府県の1ヶ月分の新規感染者数に、昨日より増えたかを表す列を追加してみましょう。

### 期待する出力フォーマット

|date| new_cases| is_incresed|
|---:|---:|---:|
|2023-01-01| 1378| False|
|2023-01-02| 1174| False|
|2023-01-03| 1318| True|


## 問8 (少し難しい)

前問で作成した結果を用いて、新規感染者数前日比増の連続日数（例: 3日連続増加）の、その期間での最大値を集計してください。

### 期待する出力フォーマット

| streak|
|---:|
|4|


### ヒント:
- Trueが連続する区間にラベル付をして、同じラベルがいくつあるか数えます。
- 連続区間のラベルは、True/Falseを区別しない通し番号と、Trueのときだけ増える番号の差を使います。


| 日付 | True/False | 通し番号 | True番号 | 差 |
|---:|---:|---:|---:|---:|
|2023-01-01|True|1|1| 0|
|2023-01-02|False|2|||
|2023-01-03|True|3|2|1|
|2023-01-04|True|4|3|1|
|2023-01-05|True|5|4|1|
|2023-01-06|False|6|

## 問9

2021年2月14日、新規感染者数が**少ない順**に10位（タイ含む）までを挙げてみましょう

この日は新規感染者数が0の県があることに注意して下さい

## 問10

2021年、毎日（365日とも）新規感染者数が1以上だった都道府県を調べてみましょう。


## 問11
北海道、東京都、神奈川県、大阪府**以外**の好きな府県を選んでください。

2021年、1番新規感染者数が多かった7日間、少なかった7日間を調べてみましょう。

(1) covid19_jpを用いて書いてみましょう。

(2) covid19_jp_unpivotted を用いて書いてみましょう。
  - ヒント: https://cloud.google.com/bigquery/docs/reference/standard-sql/array_functions?hl=ja#generate_date_array
  - https://cloud.google.com/bigquery/docs/reference/standard-sql/query-syntax?hl=ja#unnest_operator

### 期待する出力フォーマット

| start_date| end_date| sum_7days|
|---:|---:|---:|
|2021-11-20| 2021-11-27| 0|
|2021-11-21| 2021-11-28| 0|
|2021-08-17| 2021-08-24| 888|

## 問12

好きな一ヶ月を選びます。都道府県ごとの10万人あたりの感染者数の毎日の中央値を出してください。

### 期待するフォーマット
|date| median|
|---:| ---:|
|2020-01-01| 12|
|2020-01-02| 14|
|︙|︙|
